In [ ]:
import pandas as pd

r1 = pd.read_csv('r1_moisturizer.csv')
r2 = pd.read_csv('r2_moisturizer.csv')
r3 = pd.read_csv('r3_moisturizer.csv')
r4 = pd.read_csv('r4_moisturizer.csv')
r5 = pd.read_csv('r5_moisturizer.csv')

data = pd.concat([r1, r2, r3, r4, r5], ignore_index=True)


In [ ]:
data

,author_id,product_id,rating
0,33565815712,P218700,5
1,22414971528,P218700,5
2,1447823482,P218700,1
3,6772640045,P218700,1
4,13043225316,P218700,5
...,...,...,...
240775,1559233675,P501585,5
240776,6030745575,P501585,5
240777,2123263337,P481397,5
240778,1696370280,P483086,4


In [ ]:
data['author_id'].value_counts()

author_id
1696370280     48
1288462295     29
1930716686     28
1094418599     23
6758671222     23
               ..
1014808351      1
1127684398      1
5657958391      1
2706547458      1
42434587408     1
Name: count, Length: 181238, dtype: int64

In [ ]:
data.dtypes

author_id     object
product_id    object
rating         int64
dtype: object

In [ ]:
data

,author_id,product_id,rating
0,33565815712,P218700,5
1,22414971528,P218700,5
2,1447823482,P218700,1
3,6772640045,P218700,1
4,13043225316,P218700,5
...,...,...,...
240775,1559233675,P501585,5
240776,6030745575,P501585,5
240777,2123263337,P481397,5
240778,1696370280,P483086,4


In [ ]:
product=pd.read_csv("/content/product_info.csv")

In [ ]:
product=product[['product_id', 'product_name']]

In [ ]:
result = pd.merge(product[['product_id', 'product_name']], data, on='product_id', how='inner')

In [ ]:
result

,product_id,product_name,author_id,rating
0,P439055,GENIUS Sleeping Collagen Moisturizer,5880814443,5
1,P439055,GENIUS Sleeping Collagen Moisturizer,1726924575,5
2,P439055,GENIUS Sleeping Collagen Moisturizer,1551348158,5
3,P439055,GENIUS Sleeping Collagen Moisturizer,8222942765,5
4,P439055,GENIUS Sleeping Collagen Moisturizer,2403670662,2
...,...,...,...,...
240775,P430146,Superberry Hydrate + Glow Dream Oil,2079521382,5
240776,P430146,Superberry Hydrate + Glow Dream Oil,2845238877,5
240777,P430146,Superberry Hydrate + Glow Dream Oil,1364315449,5
240778,P430146,Superberry Hydrate + Glow Dream Oil,1526876973,5


In [ ]:
result['author_id'], _ = pd.factorize(result['author_id'])
result['product_id'], _ = pd.factorize(result['product_id'])

In [ ]:
result

,product_id,product_name,author_id,rating
0,0,GENIUS Sleeping Collagen Moisturizer,0,5
1,0,GENIUS Sleeping Collagen Moisturizer,1,5
2,0,GENIUS Sleeping Collagen Moisturizer,2,5
3,0,GENIUS Sleeping Collagen Moisturizer,3,5
4,0,GENIUS Sleeping Collagen Moisturizer,4,2
...,...,...,...,...
240775,437,Superberry Hydrate + Glow Dream Oil,179455,5
240776,437,Superberry Hydrate + Glow Dream Oil,181235,5
240777,437,Superberry Hydrate + Glow Dream Oil,179460,5
240778,437,Superberry Hydrate + Glow Dream Oil,181236,5


In [ ]:
!pip install pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('RecommendationSystem').getOrCreate()


In [ ]:
from pyspark.sql import DataFrame

spark_df = spark.createDataFrame(result)


In [ ]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol='author_id',
    itemCol='product_id',
    ratingCol='rating',
    nonnegative=True,
    implicitPrefs=False,
    rank=10,
    maxIter=10,
    regParam=0.1
)

model = als.fit(spark_df)


In [ ]:
# Generate top 5 product recommendations for all users
user_recommendations = model.recommendForAllUsers(5)

# Show recommendations for a specific user
user_recommendations.where(user_recommendations.author_id == 73222).show(truncate=False)

+---------+-----------------------------------------------------------------------------------+
|author_id|recommendations                                                                    |
+---------+-----------------------------------------------------------------------------------+
|73222    |[{411, 8.425037}, {217, 8.292563}, {76, 8.0921335}, {55, 8.062781}, {48, 7.962358}]|
+---------+-----------------------------------------------------------------------------------+



In [ ]:
result[result['product_id'] == 48]

,product_id,product_name,author_id,rating
32827,48,Collagen Superfusion Firming & Plumping Facial...,30425,3
32828,48,Collagen Superfusion Firming & Plumping Facial...,30426,4
32829,48,Collagen Superfusion Firming & Plumping Facial...,30427,5
32830,48,Collagen Superfusion Firming & Plumping Facial...,30428,5
32831,48,Collagen Superfusion Firming & Plumping Facial...,30429,5
32832,48,Collagen Superfusion Firming & Plumping Facial...,30430,3
32833,48,Collagen Superfusion Firming & Plumping Facial...,30431,5
32834,48,Collagen Superfusion Firming & Plumping Facial...,30432,5
32835,48,Collagen Superfusion Firming & Plumping Facial...,30433,5
32836,48,Collagen Superfusion Firming & Plumping Facial...,30434,5


In [168]:
user_recommendations.where(user_recommendations.author_id == 5).show(truncate=False)

+---------+--------------------------------------------------------------------------------------+
|author_id|recommendations                                                                       |
+---------+--------------------------------------------------------------------------------------+
|5        |[{139, 7.8158894}, {418, 7.53652}, {364, 7.3338714}, {48, 6.796111}, {217, 6.6431174}]|
+---------+--------------------------------------------------------------------------------------+

